## Домашняя работа №6

<b>Задание:</b> Реализовать адаптивный бустинг использующий Логистическую Регрессию и меру ошибок LogLoss. Сравнить с точностью адаптивного бустинга на деревьях решений. Для сбора предсказаний можно использовать ту же функцию predict что и для бустинга на деревьях<br>
<i>Примечания: в LogLoss необходимо передавать не предсказания полученные с помощью clf.predict(...), а вероятность, полученную с помощью clf.predict_proba(...)[:, 1] 
    
   Реализовать специальную функцию predict для бустинга на логистической регрессии выводящую предсказания по формуле: $ Predictions=sign(Score_{bust}) $,
где sign равен единице для положительных и нулю для отрицательных значений, а $ Score_{bust}= \sum \alpha_iScore_i$. Баллы выдаваемые каждой моделью $Score_i$ можно найти при помощи вызова метода decision_function на моделе

In [1]:
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer
from sklearn import model_selection

import matplotlib.pyplot as plt
from pprint import pprint

import warnings

In [2]:
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25)

def get_error(pred, y):
    return np.sum((pred != y).astype(int)) / len(y)

def log_loss(pred, y):
    return -np.sum(y*np.log(pred)+(1-y)*np.log(1-pred))/len(y)

def predict_proba(X):
    return 1/(1+np.exp(-1*X))

In [3]:
def adaboost(X, y, N):

    # Размер выборки
    n_objects = len(X)

    # Запишем количество классов в переменную
    n_classes = len(np.unique((y)))

    # Начальные веса деревьев
    w = np.ones(n_objects) / n_objects

    # Деревья с весами будем записывать в список
    models = []

    for n in range(N):
        # Зададим дерево и обучим его
        clf = DecisionTreeClassifier(max_depth=1)
        clf.fit(X, y, sample_weight=w)

        predictions = clf.predict(X)
        e = get_error(predictions, y)
        # отбросим дерево, если его ошибка больше 0.5
        # Запишем условие в общем виде (применимо к небинарным классификаторам)
        if e >= 1 - 1/n_classes: 
            break

        # Вычислим вес для дерева
        alpha = 0.5 * np.log((1 - e) / e)

        # Найдем индексы правильно классифицированных элементов
        match = predictions == y

        # Увеличим веса для неправильно классифицированных элементов
        w[~match] *= np.exp(alpha)

        # Нормализуем веса
        w /= w.sum()

        # Добавим дерево с весом в список
        models.append((alpha, clf))
    
    return models

In [4]:
def adaboost_logloss(X, y, N):

    # Размер выборки
    n_objects = len(X)

    # Запишем количество классов в переменную
    n_classes = len(np.unique((y)))

    # Начальные веса деревьев
    w = np.ones(n_objects) / n_objects

    # Деревья с весами будем записывать в список
    models = []

    for n in range(N):
        # Зададим дерево и обучим его
        clf = LogisticRegression()
      
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            #и здесь обучаем модели
            clf.fit(X, y, w)
    
            predictions = clf.predict_proba(X)[:, 1]
            e = log_loss(predictions, y)
            # отбросим дерево, если его ошибка больше 0.5
            # Запишем условие в общем виде (применимо к небинарным классификаторам)
            if e >= 1 - 1/n_classes: 
                break
    
            # Вычислим вес для дерева
            alpha = 0.5 * np.log((1 - e) / e)
    
            # Найдем индексы правильно классифицированных элементов
            match = (predictions > 0.5).astype(int) == y
    
            # Увеличим веса для неправильно классифицированных элементов
            w[~match] *= np.exp(alpha)
    
            # Нормализуем веса
            w /= w.sum()
    
            # Добавим дерево с весом в список
            models.append((alpha, clf, predictions))
    
    return models

In [5]:
N = 50
models = adaboost_logloss(X_train, y_train, N)
print(f'Обучим {N} деревьев\n\nСоздалось {len(models)} моделей, с alpha равными \n')
for m in models:
    print(m[0])

Обучим 50 деревьев

Создалось 17 моделей, с alpha равными 

0.9500241273522885
0.6889659234876808
0.6232658668730207
0.4819300280617255
0.35127226349851454
0.2169607178430995
0.14972052185645207
0.10613078027397012
0.08498903033363571
0.059227622248973884
0.028313372071370302
0.013400963724539118
0.007212993702055966
0.0130045602146961
0.015567202211203633
0.007918816964903739
0.02230522637292671


In [16]:
def predict(X, models):
    
    n_classes = 2
    n_objects = len(X)
    
    # вначале обозначим предсказание нулевым массивом
    y_pred = np.zeros((n_objects, n_classes))
    
    for alpha, clf, pred in models:
        prediction = clf.predict(X)
        
        # Для каждого предсказания будем прибавлять alpha к
        # элементу с индексом предсказанного класса
        y_pred[range(n_objects), prediction] += alpha
        
    # выберем индексы с максимальными суммарными весами -
    # получим предсказанные алгоритмом классы
    y_pred = np.argmax(y_pred, axis=1)
    
    return y_pred

In [17]:
def my_predict(X, models):
    n_classes = 2
    n_objects = len(X)
    
    y_pred = np.array(0)
    
    for alpha, clf, pred in models:
        
        # Получаем score умножением соответствующего alpha
        # на соответсвующий вектор предсказаний
        score = alpha * clf.decision_function(X)
        
        # Вычисляем суммарный вектор
        y_pred = y_pred + score
        
    for num, s in enumerate(y_pred):
        
        # Если больше 0, то 1, если меньше - 0.
        if s >= 0:
            y_pred[num] = 1
        else: 
            y_pred[num] = 0
            
    return y_pred

In [25]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print(f'Точность первого алгоритма на обучающей выборке: {(1 - get_error(predict(X_train, models), y_train)) * 100:.3f}')
    print(f'Точность второго алгоритма на обучающей выборке: {(1 - get_error(my_predict(X_train, models), y_train)) * 100:.3f}')
    print(f'Точность первого алгоритма на тестовой выборке с LogLoss: {(1 - log_loss(predict_proba(predict(X_test, models)), y_test)) * 100:.3f}')
    print(f'Точность второго алгоритма на тестовой выборке с LogLoss: {(1 - log_loss(predict_proba(my_predict(X_test, models)), y_test)) * 100:.3f}')

Точность первого алгоритма на обучающей выборке: 93.427
Точность второго алгоритма на обучающей выборке: 94.131
Точность первого алгоритма на тестовой выборке с LogLoss: 51.168
Точность второго алгоритма на тестовой выборке с LogLoss: 51.434


Почему такая точность у LogLoss не понятно...